In [4]:
import pandas as pd
import numpy as np
import statsmodels as sms
import statsmodels.formula.api as sm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

### 1. In this problem, you will use the Carseats data set attached in the assignment (Carseats.csv) for linear regression.
### (a) [5 pts] Fit a multiple linear regression model to predict Sales using Price, Urban, and US. Report the R2 of the model.

In [79]:
# Import CSV file
carseats_data = pd.read_csv('Carseats.csv')
# Replace 'Yes' and 'No' cells into 1 and 0 each.
carseats_data['US'] = carseats_data['US'].replace(['Yes'],1)
carseats_data['US'] = carseats_data['US'].replace(['No'],0)
carseats_data['Urban'] = carseats_data['Urban'].replace(['Yes'],1)
carseats_data['Urban'] = carseats_data['Urban'].replace(['No'],0)
result = sm.ols('Sales ~ Price + Urban + US',data = carseats_data).fit()
print(result.summary())
# X = np.column_stack((carseats_data['Price'], carseats_data['Urban'], carseats_data['US']))
# Y =carseats_data['Sales']
# X2 = sm.add_constant(X)
# prob1_result = sm.OLS(Y, X2)
# prob1_result = prob1_result.fit()
# print(prob1_result.summary())


# # Only interested in several columns
# A=carseats_data[['Sales', 'Price', 'Urban', 'US']]
# lin_variables = np.array(A.values,'int')
# # Split into predictors and result variables.
# Y_lin_variables=lin_variables[:,0]
# X_ones=np.ones((Y_lin_variables.size,1),dtype=int)
# X_lin_variables=np.column_stack([X_ones,pd.DataFrame(lin_variables[:,1:4])])
# # The Least Square Estimator
# X_coefficient= np.matmul(np.linalg.inv(np.matmul(np.transpose(X_lin_variables),X_lin_variables)), np.transpose(X_lin_variables))
# beta=np.matmul(X_coefficient,Y_lin_variables)
# print("------------------------------------------------------")
# print("a multiple linear regression model to predict Sales using Price, Urban, and US is ")
# print("(",beta[0],") + (",beta[1],")*","Price"," + (",beta[2],") * Urban + (",beta[3],") * US")
# print("------------------------------------------------------")
# # Finding R-squared
# Y_lin_predict=np.dot(X_lin_variables, beta)
# rss=np.sum(np.square(Y_lin_variables-Y_lin_predict))
# tss=np.sum(np.square((Y_lin_variables-np.mean(Y_lin_variables))))
# r_squared=1-(rss/tss)
# print("R-squared of the model is")
# print(r_squared)
# print("------------------------------------------------------")



                            OLS Regression Results                            
Dep. Variable:                  Sales   R-squared:                       0.239
Model:                            OLS   Adj. R-squared:                  0.234
Method:                 Least Squares   F-statistic:                     41.52
Date:                Sun, 03 Apr 2022   Prob (F-statistic):           2.39e-23
Time:                        15:34:03   Log-Likelihood:                -927.66
No. Observations:                 400   AIC:                             1863.
Df Residuals:                     396   BIC:                             1879.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     13.0435      0.651     20.036      0.0

### (b) [10 pts] Provide an interpretation of each coefficient in the model. Be careful — some of the variables in the model are qualitative

- The intercept, 13.0435, refers to the sales on baseline, where price is 0 and the value to 'urban' and 'US' is 'NO'
- A unit change in price is associated with a -0.0544 change in Sales, while all variables stay fixed. It is supported by the fact that p-value is extremely small.
- Living in urban area will have as negative effect as -0.0219 on sales. However, large p-value suggests no statistical evidence of a real difference on sales by urban variable.
- Living in US will have as positive effect as 1.2006 on sales. It is statistically supported by small p-value

### (c) [5 pts] Write out the model in equation form, being careful to handle the qualitative variables properly.

In [62]:
beta=[13.0435,-0.0545,-0.0219,1.2006]
print("a multiple linear regression model to predict Sales using Price, Urban, and US is ")
print("(",beta[0],") + (",beta[1],")*","Price"," + (",beta[2],") * Urban + (",beta[3],") * US")

a multiple linear regression model to predict Sales using Price, Urban, and US is 
( 13.0435 ) + ( -0.0545 )* Price  + ( -0.0219 ) * Urban + ( 1.2006 ) * US


### (d) [5 pts] For which predictor variable j can you reject the null hypothesis H0 : βj =0?

Only Price and US can reject the null hypothesis based on p-values.

### (e) [5 pts] On the basis of your response to the previous question, fit a smaller model that only uses the predictors for which there is evidence of association with the outcome.

In [81]:
result = sm.ols('Sales ~ Price  + US',data = carseats_data).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  Sales   R-squared:                       0.239
Model:                            OLS   Adj. R-squared:                  0.235
Method:                 Least Squares   F-statistic:                     62.43
Date:                Sun, 03 Apr 2022   Prob (F-statistic):           2.66e-24
Time:                        15:34:31   Log-Likelihood:                -927.66
No. Observations:                 400   AIC:                             1861.
Df Residuals:                     397   BIC:                             1873.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     13.0308      0.631     20.652      0.0

### (f) [10 pts] How well do the models in Part (a) and Part (e) fit the data? Compare the two models. Can you say one model is better than the other based on R2? Provide justification for your answer.

Both models have the same R-squared value, which means we cannot compare both based on R-squared.
However, a larger f-value may note that the model in part (e) fit the data much better than that in part (a).

### (g) [10 pts] Using the model from (e), obtain 95% confidence intervals for the coefficient(s).

In [82]:
result.conf_int(alpha=0.05, cols=None)

,0,1
Intercept,11.79032,14.271265
Price,-0.06476,-0.044195
US,0.69152,1.707766


### 2. [50 pts] In class, we used the example of the logistic regression model to predict the probability of default using income and balance on the Default data set attached in the assignment (Default.csv). In this problem, we will estimate the test error of this logistic regression model using the validation set approach. Do not forget to set a random seed before beginning your analysis.
### (a) [10 pts] Fit a logistic regression model that uses income and balance to predict default using the validation set approach, and estimate the test error of this model. In order to do this, you must perform the following steps:
- i Split the sample set into a training set and a validation set.
- ii Fit a multiple logistic regression model using only the training observations.
- iii Obtain a prediction of default status for each individual in the validation set by computing the posterior probability of default for that individual, and classifying the individual to the default category if the posterior probability is greater than 0.5.
- iv Compute the validation set error, which is the fraction of the observations in
the validation set that are misclassified.

In [101]:
from tabulate import tabulate

# split into a training set and validation set
default_data = pd.read_csv('Default.csv')
default_data['default'] = default_data['default'].replace(['Yes'],1)
default_data['default'] = default_data['default'].replace(['No'],0)
default_data['student'] = default_data['student'].replace(['Yes'],1)
default_data['student'] = default_data['student'].replace(['No'],0)
train, test = train_test_split(default_data, test_size=0.5)

X_train=train[["income","balance"]]
Y_train=train["default"]
X_test=test[["income","balance"]]
Y_test=test["default"]

# Fit a model using training data
model = LogisticRegression()
model.fit(X_train,Y_train)
print(model.intercept_)
print(model.coef_)

#Predict each individual from test data
P_test=pd.DataFrame(model.predict_proba(X_test), index=X_test.index)
P_test = pd.DataFrame((model.predict_proba(X_test)[:,1] >= 0.5).astype(int), index=X_test.index)
P_test.columns = ["predicted default"]
P_test_predict=pd.merge(test, P_test, left_index=True, right_index=True)
# Finding Test Error
result_table= [[0,0,0],[0,0,0],[0,0,0]]
for index, row in P_test_predict.iterrows():
    if row["predicted default"]==0:
        if row["default"] ==0 :
            result_table[0][0]+=1
            result_table[2][0]+=1
        elif row["default"] ==1:
            result_table[0][1]+=1
            result_table[2][1]+=1
        result_table[0][2]+=1
    elif row["predicted default"]==1:
        if row["default"] ==0:
            result_table[1][0]+=1
            result_table[2][0]+=1
        elif row["default"] ==1:
            result_table[1][1]+=1
            result_table[2][1]+=1
        result_table[1][2]+=1
    result_table[2][2]+=1
result_df= pd.DataFrame(result_table)
result_df.columns=[0,1,2]
print(result_df)
print("False Positive Rate is",round( result_table[1][0]/result_table[2][0],4))
print("False Negative Rate is",round( result_table[0][1]/result_table[2][1],4))




[-11.86423424]
[[2.07470644e-05 5.85629664e-03]]
      0    1     2
0  4809  116  4925
1    26   49    75
2  4835  165  5000
False Positive Rate is 0.0054
False Negative Rate is 0.703


### (b) [10 pts] Provide an interpretation of each coefficient in the trained model.

In [264]:
X_train=sms.tools.tools.add_constant(X_train)
model= sms.api.Logit(Y_train, X_train).fit()
print(model.summary())

Optimization terminated successfully.
         Current function value: 0.074226
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:                default   No. Observations:                 5000
Model:                          Logit   Df Residuals:                     4997
Method:                           MLE   Df Model:                            2
Date:                Sun, 03 Apr 2022   Pseudo R-squ.:                  0.4491
Time:                        21:27:35   Log-Likelihood:                -371.13
converged:                       True   LL-Null:                       -673.71
Covariance Type:            nonrobust   LLR p-value:                3.895e-132
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -11.5465      0.636    -18.145      0.000     -12.794     -10.299
income      2.213e-05   7.28

Log odds can be represented as log(p/(1-p))=(-11.54652535)+(2.213e-05)*income+(0.0055)*balance.
That is, a unit increase in income is associated with an increase in the log odds of default by 2.213e-05 units.
Also, a unit increase in balance is associated with an increase in the log odds of default by 0.0055 units.
Above all are supported by a small p-value

### (c) [15 pts] Perform 5-fold cross-validation using the same model as in Part (a). Write your own code for the K-fold cross-validation. How does the validation error of the cross-validation the differ from to the results in Part (a)?

In [108]:
default_data_randomized=default_data.sample(frac=1) # shuffle raw default data
misclassified=0
for count in range(5):
    train = default_data_randomized.iloc[2000*count:2000*(count+1),:]
    test= pd.concat([default_data_randomized.iloc[:2000*count,:],default_data_randomized.iloc[2000*(count+1):,:]])


    X_train=train[["income","balance"]]
    Y_train=train["default"]
    X_test=test[["income","balance"]]
    Y_test=test["default"]
    model = LogisticRegression()
    model.fit(X_train,Y_train)
    P_test=pd.DataFrame(model.predict_proba(X_test), index=X_test.index)
    P_test = pd.DataFrame((model.predict_proba(X_test)[:,1] >= 0.5).astype(int), index=X_test.index)
    P_test.columns = ["predicted default"]
    P_test_predict=pd.merge(test, P_test, left_index=True, right_index=True)
    print("iteration #: ", count+1)
    result_table= [[0,0,0],[0,0,0],[0,0,0]]
    for index, row in P_test_predict.iterrows():
        if row["predicted default"]==0:
            if row["default"] ==0 :
                result_table[0][0]+=1
                result_table[2][0]+=1
            elif row["default"] ==1:
                result_table[0][1]+=1
                result_table[2][1]+=1
            result_table[0][2]+=1
        elif row["predicted default"]==1:
            if row["default"] ==0:
                result_table[1][0]+=1
                result_table[2][0]+=1
            elif row["default"] ==1:
                result_table[1][1]+=1
                result_table[2][1]+=1
            result_table[1][2]+=1
        result_table[2][2]+=1
    result_df= pd.DataFrame(result_table)
    result_df.columns=[0,1,2]
    misclassified+=result_table[1][0]+result_table[0][1]
    print(result_df)
    print("False Positive Rate is",round( result_table[1][0]/result_table[2][0],4))
    print("False Negative Rate is",round( result_table[0][1]/result_table[2][1],4))
print("Overall # of misclassified data is ",misclassified)

iteration #:  1
      0    1     2
0  7727  260  7987
1    12    1    13
2  7739  261  8000
False Positive Rate is 0.0016
False Negative Rate is 0.9962
iteration #:  2
      0    1     2
0  7712  199  7911
1    19   70    89
2  7731  269  8000
False Positive Rate is 0.0025
False Negative Rate is 0.7398
iteration #:  3
      0    1     2
0  7682  194  7876
1    37   87   124
2  7719  281  8000
False Positive Rate is 0.0048
False Negative Rate is 0.6904
iteration #:  4
      0    1     2
0  7712  165  7877
1    36   87   123
2  7748  252  8000
False Positive Rate is 0.0046
False Negative Rate is 0.6548
iteration #:  5
      0    1     2
0  7707  190  7897
1    24   79   103
2  7731  269  8000
False Positive Rate is 0.0031
False Negative Rate is 0.7063
Overall misclassified data is  1136


The validation error is much better in Part (a) compared to part (c). This is because less data was made use of in part (c)

### (d) [15 pts] Now consider a logistic regression model that predicts the probability of default using income, balance, and a dummy variable for student. Estimate the test error for this model using the 5-fold cross-validation set approach. Comment on whether or not including a dummy variable for student would lead to a reduction in the test error rate.

In [109]:
# split into a training set and validation set
default_data = pd.read_csv('Default.csv')
default_data['default'] = default_data['default'].replace(['Yes'],1)
default_data['default'] = default_data['default'].replace(['No'],0)
default_data['student'] = default_data['student'].replace(['Yes'],1)
default_data['student'] = default_data['student'].replace(['No'],0)
default_data_randomized=default_data.sample(frac=1) # shuffle raw default data
misclassified=0

for count in range(5):
    train = default_data_randomized.iloc[2000*count:2000*(count+1),:]
    test= pd.concat([default_data_randomized.iloc[:2000*count,:],default_data_randomized.iloc[2000*(count+1):,:]])
    X_train=train[["income","balance","student"]]
    Y_train=train["default"]
    X_test=test[["income","balance","student"]]
    Y_test=test["default"]
    # Fit a model using training data
    model = LogisticRegression()
    model.fit(X_train,Y_train)
    P_test=pd.DataFrame(model.predict_proba(X_test), index=X_test.index)
    P_test = pd.DataFrame((model.predict_proba(X_test)[:,1] >= 0.5).astype(int), index=X_test.index)
    P_test.columns = ["predicted default"]
    P_test_predict=pd.merge(test, P_test, left_index=True, right_index=True)
    print("iteration #: ", count+1)
    result_table= [[0,0,0],[0,0,0],[0,0,0]]
    for index, row in P_test_predict.iterrows():
        if row["predicted default"]==0:
            if row["default"] ==0 :
                result_table[0][0]+=1
                result_table[2][0]+=1
            elif row["default"] ==1:
                result_table[0][1]+=1
                result_table[2][1]+=1
            result_table[0][2]+=1
        elif row["predicted default"]==1:
            if row["default"] ==0:
                result_table[1][0]+=1
                result_table[2][0]+=1
            elif row["default"] ==1:
                result_table[1][1]+=1
                result_table[2][1]+=1
            result_table[1][2]+=1
        result_table[2][2]+=1
    result_df= pd.DataFrame(result_table)
    result_df.columns=[0,1,2]
    misclassified+=result_table[1][0]+result_table[0][1]
    print(result_df)
    print("False Positive Rate is",round( result_table[1][0]/result_table[2][0],4))
    print("False Negative Rate is",round( result_table[0][1]/result_table[2][1],4))
print("Overall # of misclassified data is ",misclassified)



iteration #:  1
      0    1     2
0  7689  207  7896
1    50   54   104
2  7739  261  8000
False Positive Rate is 0.0065
False Negative Rate is 0.7931
iteration #:  2
      0    1     2
0  7695  223  7918
1    39   43    82
2  7734  266  8000
False Positive Rate is 0.005
False Negative Rate is 0.8383
iteration #:  3
      0    1     2
0  7697  220  7917
1    38   45    83
2  7735  265  8000
False Positive Rate is 0.0049
False Negative Rate is 0.8302
iteration #:  4
      0    1     2
0  7708  234  7942
1    27   31    58
2  7735  265  8000
False Positive Rate is 0.0035
False Negative Rate is 0.883
iteration #:  5
      0    1     2
0  7697  198  7895
1    28   77   105
2  7725  275  8000
False Positive Rate is 0.0036
False Negative Rate is 0.72
Overall misclassified data is  1264


- Including a dummy variable, student, may not lead to a reduction in the test error rate